In [15]:
import sqlite3
import pandas as pd

df = pd.read_csv("../data/processed/bf_efficiency_clean.csv")

conn = sqlite3.connect("../data/bf.db")
df.to_sql("line_trials", conn, if_exists="replace", index=False)
conn.close()


In [16]:
conn = sqlite3.connect("../data/bf.db")

query = """
SELECT line_id,
       AVG(efficiency) AS avg_efficiency
FROM line_trials
GROUP BY line_id
ORDER BY avg_efficiency DESC;
"""

pd.read_sql(query, conn)

,line_id,avg_efficiency
0,TML BRO 16.6.1,1814.583333
1,TML BRO 16.6.2,1697.916667
2,TML BRO 16.6.3,1525.000000
3,TML BRO 16.4.6,1327.083333
4,TML BRO 16.4.5,1300.000000
...,...,...
135,TML AMP 15.3.1,207.916667
136,TML AMP 15.2.2,190.000000
137,TML AMP 15.2.1,137.708333
138,LAO BRO 20.5.5,106.250000


In [17]:
query = """
SELECT
    COUNT(*) AS total_trials,
    AVG(blood_input_ml) AS avg_input,
    AVG(egg_output_total) AS avg_output,
    AVG(efficiency) AS avg_efficiency,
    AVG(eggs_per_hour) AS avg_output_rate
FROM line_trials;
"""

pd.read_sql(query, conn)



,total_trials,avg_input,avg_output,avg_efficiency,avg_output_rate
0,140,2397.714286,1.741887e+06,719.449977,36289.315476


In [18]:
query = """
SELECT
    CAST(blood_input_ml / 500 AS INT) * 500 AS input_bucket,
    COUNT(*) AS n_trials,
    AVG(efficiency) AS avg_efficiency,
    AVG(efficiency * efficiency) - AVG(efficiency) * AVG(efficiency) AS variance
FROM line_trials
GROUP BY input_bucket
ORDER BY input_bucket;
"""

df_sql = pd.read_sql(query, conn)
df_sql

,input_bucket,n_trials,avg_efficiency,variance
0,0,2,73.333333,1083.506944
1,500,1,486.250000,0.000000
2,1500,3,700.625000,11459.375000
3,2000,120,749.287126,73919.009496
4,3000,14,576.696429,7869.624920


In [19]:
conn.close()
